<a href="https://colab.research.google.com/github/Snowdenstyll/keno/blob/main/jupyterNotebooks/Keno_Lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [44]:
df = pd.read_csv("2024.csv")

In [45]:
df.head()
df.tail()
df.info()
df.describe()
df.drop(['PlayDate', 'AP'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PlayDate  248 non-null    object
 1   AP        248 non-null    object
 2   N01       248 non-null    int64 
 3   N02       248 non-null    int64 
 4   N03       248 non-null    int64 
 5   N04       248 non-null    int64 
 6   N05       248 non-null    int64 
 7   N06       248 non-null    int64 
 8   N07       248 non-null    int64 
 9   N08       248 non-null    int64 
 10  N09       248 non-null    int64 
 11  N10       248 non-null    int64 
 12  N11       248 non-null    int64 
 13  N12       248 non-null    int64 
 14  N13       248 non-null    int64 
 15  N14       248 non-null    int64 
 16  N15       248 non-null    int64 
 17  N16       248 non-null    int64 
 18  N17       248 non-null    int64 
 19  N18       248 non-null    int64 
 20  N19       248 non-null    int64 
 21  N20       248 no

In [46]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [47]:
# All our games
number_of_rows = df.values.shape[0]
# Amount of games we need to take into consideration for prediction
window_length = 10
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)


In [48]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [49]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
#batch_size = 40
batch_size = 60

# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

In [50]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=70)

# Train the model
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=400, verbose=2)
#history = model.fit(X_train, y_train, batch_size=100, epochs=600, verbose=2)

Epoch 1/400
2/2 - 19s - loss: 1.1256 - accuracy: 0.0704 - 19s/epoch - 9s/step
Epoch 2/400
2/2 - 0s - loss: 1.1171 - accuracy: 0.0704 - 58ms/epoch - 29ms/step
Epoch 3/400
2/2 - 0s - loss: 1.1106 - accuracy: 0.0141 - 55ms/epoch - 28ms/step
Epoch 4/400
2/2 - 0s - loss: 1.1049 - accuracy: 0.0563 - 55ms/epoch - 27ms/step
Epoch 5/400
2/2 - 0s - loss: 1.0992 - accuracy: 0.0563 - 55ms/epoch - 27ms/step
Epoch 6/400
2/2 - 0s - loss: 1.0931 - accuracy: 0.0563 - 48ms/epoch - 24ms/step
Epoch 7/400
2/2 - 0s - loss: 1.0854 - accuracy: 0.0563 - 48ms/epoch - 24ms/step
Epoch 8/400
2/2 - 0s - loss: 1.0797 - accuracy: 0.0704 - 48ms/epoch - 24ms/step
Epoch 9/400
2/2 - 0s - loss: 1.0758 - accuracy: 0.0563 - 47ms/epoch - 24ms/step
Epoch 10/400
2/2 - 0s - loss: 1.0709 - accuracy: 0.0845 - 46ms/epoch - 23ms/step
Epoch 11/400
2/2 - 0s - loss: 1.0612 - accuracy: 0.0423 - 44ms/epoch - 22ms/step
Epoch 12/400
2/2 - 0s - loss: 1.0543 - accuracy: 0.0986 - 43ms/epoch - 21ms/step
Epoch 13/400
2/2 - 0s - loss: 1.0458 - 

In [51]:
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

8/8 [==============================] - 2s 12ms/step - loss: 1.2732 - accuracy: 0.2437
Test Loss: 1.273168921470642, Test Accuracy: 0.24369747936725616
6/6 [==============================] - 0s 12ms/step - loss: 1.7950 - accuracy: 0.0539
Test Loss: 1.795026421546936, Test Accuracy: 0.053892213851213455


In [52]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

In [53]:
''' import datetime
import pytz

# Get the current time in a specific timezone
tz = pytz.timezone('America/New_York')
current_day = datetime.datetime.now(tz)

time_label = "Midday" if (current_day.hour <= 14) else "Evening"

label = current_day.strftime("%Y_%m_%d_") + time_label
model.save(f'Models/{label}.keras')
print("saved model") '''

' import datetime\nimport pytz\n\n# Get the current time in a specific timezone\ntz = pytz.timezone(\'America/New_York\')\ncurrent_day = datetime.datetime.now(tz)\n\ntime_label = "Midday" if (current_day.hour <= 14) else "Evening"\n\nlabel = current_day.strftime("%Y_%m_%d_") + time_label\nmodel.save(f\'Models/{label}.keras\')\nprint("saved model") '

In [54]:
df1 = df.copy()
to_predict = df1.tail(window_length)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-54-efb52fef0daf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
238,2,3,5,7,10,20,26,28,36,37,40,42,45,47,48,51,52,58,60,64
239,2,9,14,15,17,18,21,25,29,34,38,39,40,58,59,63,64,66,69,70
240,1,7,8,10,16,20,23,24,26,29,30,38,49,52,54,62,63,66,67,68
241,1,2,10,12,16,21,25,26,27,28,29,31,33,35,43,51,53,54,62,66
242,1,5,7,10,12,18,25,31,32,33,37,40,41,43,47,49,54,55,56,61
243,7,8,9,16,19,22,23,29,30,37,43,46,50,52,54,58,61,62,64,68
244,2,9,14,16,19,22,24,25,26,27,30,33,34,45,51,56,57,60,62,67
245,3,5,7,15,24,26,27,28,29,33,38,39,41,43,55,56,57,59,60,70
246,1,10,12,13,15,17,19,20,28,34,37,41,42,45,50,52,62,64,67,69


In [78]:
prediction = np.array(df.tail(1))

x_next = scaler.transform(to_predict)
y_pred = model.predict(np.array([x_next]))

temp = np.array(np.unique(scaler.inverse_transform(y_pred).astype(int)[0]))
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[uniq_idx[counts>=2] ]  # <--- Get duplicates

top_5_indices = np.argsort(y_pred[0], axis=0)[::-1][:5]

top_5_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

# Print the winning indices

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices:", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")
top_10_indices = np.argsort(y_pred[0], axis=0)[::-1][:10]
top_10_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_10_numbers.append(number)

print("The top 10 indices:", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("The predicted numbers for the lottery game are:  ", temp)
print("The actual numbers in the last lottery game were:", prediction[0])
print("\n")
print("The winning indices:", winning_indices)
print("winners:", duplicates)
print(len(duplicates)/20)


1/1 [==============================] - 0s 24ms/step
The top 5 indices: [17 15 14  5 18]
The top 5 numbers: [62, 53, 49, 17, 64]


The top 10 indices: [17 15 14  5 18 16  0  6 19  7]
The top 10 numbers: [62, 53, 49, 17, 64, 56, 2, 19, 66, 25]


The predicted numbers for the lottery game are:   [ 2  4  5 10 14 19 22 24 27 29 31 35 40 43 49 53 56 60 62 65]
The actual numbers in the last lottery game were: [ 2  5  6  9 10 17 19 25 26 27 28 30 40 42 49 53 56 62 64 66]


The winning indices: [ 0  1  4  6  9 12 14 15 16 17]
winners: [ 2  5 10 19 27 40 49 53 56 62]
0.5


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


# Predict the next set of numbers

In [79]:
next = df.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
prediction = scaler.inverse_transform(y_next_pred).astype(int)
print("The predicted numbers for the lottery game:", temp)
print("\n")
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]
top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)

top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_10_numbers = []
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)
print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)


1/1 [==============================] - 0s 23ms/step
The predicted numbers for the lottery game: [ 2  4  5 10 14 19 22 24 27 29 31 35 40 43 49 53 56 60 62 65]


The top 5 indices [14  0  5 15 17]
The top 5 numbers: [51, 3, 20, 53, 59]


The top 10 indices [14  0  5 15 17 12  6 13  4 16]
The top 10 numbers: [51, 3, 20, 53, 59, 42, 22, 45, 15, 55]


winning indices [ 0  1  4  6  9 12 14 15 16 17]
winning numbers [3, 4, 15, 22, 31, 42, 51, 53, 55, 59]


In [80]:
next = df.copy()
next = next.tail(5)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))

# Get the actual values of the top 10 predictions
prediction = scaler.inverse_transform(y_next_pred).astype(int)

# Get the indices of the top 10 predictions (change axis=0 if predictions are along a different axis)
top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]

top_10_numbers = []
top_5_numbers = []

# index [0,2,4] usually are correct
# Loop through each index in top_10_indices
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_5_numbers.append(number)

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)


print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)
print("\n")

print("The predicted numbers:", prediction)


1/1 [==============================] - 0s 22ms/step
The top 5 indices [15 14 16  0 13]
The top 5 numbers: [54, 50, 57, 3, 47]


The top 10 indices [15 14 16  0 13 17  8  7  6  5]
The top 10 numbers: [54, 50, 57, 3, 47, 60, 29, 26, 23, 19]


winning indices [ 0  1  4  6  9 12 14 15 16 17]
winning numbers [3, 6, 16, 23, 32, 43, 50, 54, 57, 60]


The predicted numbers: [[ 3  6  9 13 16 19 23 26 29 32 35 39 43 47 50 54 57 60 62 66]]
